# Audio recording

With the help of this notebook you can populate the word dataset.

In [1]:
import pyaudio
import wave
import keyboard
import threading as th
import time
import os
import glob
from uuid import uuid4
import shutil

In [2]:
def create_directory(dir_path: str):
    """Creates a directory if it is none existent."""
    isExist = os.path.exists(dir_path)
    if not isExist:
        os.makedirs(dir_path)
        print("The new directory is created!")
        return
    print("Directory already created.")

In [13]:
print("Deleting tmp content")

tmp_dir = "data/tmp"
for path in os.listdir(tmp_dir):
    if path == ".ipynb_checkpoints":
        continue
    full_path = os.path.join(tmp_dir,path)
    os.remove(full_path)

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 16000  # Record at 16000 samples per second

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print("Enter the world you will say.")
word = input()


print("Press 's' key to start recording")
print("Press 'q' key to quit recording")
while(True):
    if keyboard.is_pressed("s"):
        break
    
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames
import threading as th

keep_going = True
def key_capture_thread():
    global keep_going
    while(True):
        if keyboard.is_pressed("q"):        
            keep_going = False
            break
        time.sleep(0.01)

th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()

print("Starting audio recording.")
while keep_going:
    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk)):
        data = stream.read(chunk)
        frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording. Saving audio.')
filename=os.path.join(tmp_dir,"output.wav")


# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()


own_recordings_dir = "data/own_recordings"
create_directory(own_recordings_dir)
own_recordings_dir_class = os.path.join(own_recordings_dir,word)
create_directory(own_recordings_dir_class)
print("Splitting into parts")
!python pyAudioAnalysis/pyAudioAnalysis/audioAnalysis.py silenceRemoval -i data/tmp/output.wav --smoothing 1.0 --weight 0.3

#Copying the recording parts into an organized folder
for path in os.listdir(tmp_dir):
    if path=='output.wav' or path==".ipynb_checkpoints":
        continue
    print(path)
    id_ = str(uuid4())+".wav"
    orig_full_path = os.path.join(tmp_dir,path)
    new_full_path = os.path.join(tmp_dir,id_)
    os.rename(orig_full_path,new_full_path)
    shutil.copy2(new_full_path,own_recordings_dir_class)

Deleting tmp content
Enter the world you will say.


 cat


Press 's' key to start recording
Press 'q' key to quit recording
Starting audio recording.
Finished recording. Saving audio.
Directory already created.
Directory already created.
Splitting into parts
Figure(640x480)
output_0.800-1.800.wav
output_2.050-3.600.wav
output_4.450-6.050.wav
output_6.850-8.150.wav


C:\Users\zsomb\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
